In [1]:
import csv
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize,word_tokenize

In [2]:
#function that loads a lexicon of positive words to a set and returns the set
def loadLexicon(fname):
    newLex=set()
    lex_conn=open(fname)
    
    #add every word in the file to the set
    for line in lex_conn:
        newLex.add(line.strip())# remember to strip to remove the lin-change character
    lex_conn.close()

    return newLex

def review_tokenize(text):
    positive_words = loadLexicon(r"positive-words.txt")
    negative_words = loadLexicon(r"negative-words.txt")
    
    df = pd.DataFrame(columns = ('word', 'rating')) # create dataframe for words ant there ratings

    sentences=sent_tokenize(text) # split the review into sentences    
    for sentence in sentences: # for each sentence
        words=word_tokenize(sentence)  # split the review into words
        tagged_words=nltk.pos_tag(words) # POS tagging for the words in the sentence
        prev_word = [None,None]
        for tagged_word in tagged_words:  # for each word
            rating = 0 # put rating 0
            if len(tagged_word[0]) < 3: continue # ignore nouns with less than 3 characters
            if tagged_word[1] == 'NN': # if word is noun                
                add = False # mark if add word to dataframe or not
                for i in range(0, len(tagged_words)):
                    if tagged_words[i][0].lower() in negative_words: # -1 rating if there is negative word
                        if i > 0 and tagged_words[i-1][0].lower() == 'not':
                            rating += 1
                        else:
                            rating -= 1
                        add = True
                    if tagged_words[i][0].lower() in positive_words: #  +1 rating if there is negative word
                        if i > 0 and tagged_words[i-1][0].lower() == 'not':
                            rating -= 1
                        else:
                            rating += 1
                        add = True
                if add: #add word to dataframe
                    df = df.append({'word': tagged_word[0].lower(), 'rating': rating}, ignore_index=True)

    df = df.groupby('word').sum() #groupby by words
    return df
        

,rating
word,
day,1


In [6]:
def parse(input_file, num1, num2):
    
    csv_file = pd.read_csv(input_file, names=['text', 'rating']) # read csv file
    
    text1 = csv_file.iloc[num1]['text'] # read review 1
    text2 = csv_file.iloc[num2]['text'] # read review 2
    
    df1 = review_tokenize(text1) # preparing review 1 for comparison
    df2 = review_tokenize(text2) # preparing review 2 for comparison
    
    df_merged = df1.merge(df2, how='inner', on='word').reset_index() # merge dataframe
    result = df_merged.loc[~((df_merged['rating_x'] > 0) & (df_merged['rating_y'] > 0)) | 
                           ((df_merged['rating_x'] < 0) & (df_merged['rating_y'] < 0))]['word'].tolist() # choose nouns with different sentiments
    
    print(df_merged)  # dataframe of words from each review, x_rating is rating of review 1 and y_rating is rating of review 2
    
    return result
    

In [7]:
parse('amazonreviews.csv', 9, 12)

       word  rating_x  rating_y
0       app        -1         2
1      bass         2         3
2       bit         1         2
3       ear        -1         3
4     music         1         0
5     noise       -11         1
6      pair         4         1
7     phone         1         2
8     price         1         1
9     punch        -2         3
10  quality         2         5
11    sound         0         2
12      use         1         1


['app', 'ear', 'music', 'noise', 'punch', 'sound']